In [ ]:
class base_walker: 
    def __init__(self):
        """
        this class is the parent class base_walker 
        and generates aa sequences per random_walker object 
        """
        pass
    def one_step(self):
        pass
    def steps(self, seq):
        """
        seq : aa sequence from previous iteration 
          
        return none
        """
        for char in seq:
            self.one_step(char)
            
# one solution:
class random_walker(base_walker):
    def __init__(self, freq_dict):
        """
        this class inherits the abstract parent class base_walker 
        and generates aa per random_walker object 
        """
        # define stored variables, initial data
        self.x = "" #step, assumes that x = 0 at the start of the random walk.
        self.walk = self.x #list of random steps in a walk 
        self.prob = freq_dict 
        
    def one_step(self, aa):
        """
        this function determines whether a substitution events will 
        occur and if it does it generates a random aa else returns 
        the original aa, it then appends the aa to the remaining current
        sequence
        
        Return None
        """
        temp_dict = self.prob #store temp dictionary, keep original unchanged
      
        new_keys = list(temp_dict.keys()) #store keys
        new_values = list(temp_dict.values())#store values
        
        event = np.random.choice(['s', 'c'],p=[0.24,0.76]) #(Shah et al. 2020) mutation prob
        if event == 's' and aa != "[YRNPFEDIS]": #if a substituition event
            if aa == 'Y': #(Jia and Gong 2021) mutation prob
                aa = 'F'
            elif aa == 'S':#(Jia and Gong 2021) mutation prob
                aa = 'N'
            else: 
                self.x = np.random.choice(new_keys, p=new_values) #use freq of aa as probabilties 
        elif event == 'c': #if no substitution event, then return same aa 
            self.x = aa
            
        self.walk += self.x
    def get_walk(self):
        """
        Returns a list of the positions(scoring) of one walk 
        """
        # returns the history of the position of the walker
        return self.walk     